# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv('weather_raw_data.csv')
df

,City,Latitude,Longitude,Max Temp (F),Humidity,Cloud Coverage,Wind Speed,Country,Datetime
0,jamestown,42.0970,-79.2353,40.64,63.0,0,8.05,US,1648187257
1,tateyama,34.9833,139.8667,58.08,55.0,25,10.20,JP,1648187258
2,talnakh,69.4865,88.3972,15.21,91.0,100,12.41,RU,1648187260
3,lebu,-37.6167,-73.6500,54.79,80.0,1,20.98,CL,1648187261
4,koumra,8.9126,17.5539,82.35,42.0,75,2.26,TD,1648187262
...,...,...,...,...,...,...,...,...,...
545,piopio,-38.4667,175.0167,58.66,81.0,99,8.12,NZ,1648188412
546,bodden town,19.2833,-81.2500,80.01,74.0,36,7.00,KY,1648188413
547,saint-francois,46.4154,3.9054,31.44,93.0,1,2.30,FR,1648188414
548,kaitong,44.8135,123.1500,50.11,29.0,69,14.99,CN,1648188415


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitiude and longitude in locations variable
locations = df[['Latitude', 'Longitude']]
# Store humidty data as the weight
humidity = df['Humidity']
# Configure gmap
gmaps.configure(api_key=g_key)
figure = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, point_radius=2,
                                 max_intensity=100)
                                 
# # Add heat map layer to fig with .add_layer() function
figure.add_layer(heat_layer)
# # Display figure
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_df = df[(df['Max Temp (F)'] > 65) & 
              (df['Max Temp (F)'] < 90) & 
              (df['Humidity'] < 80) & 
              (df['Cloud Coverage'] < 10) &
              (df['Wind Speed'] < 5)]
ideal_df

,City,Latitude,Longitude,Max Temp (F),Humidity,Cloud Coverage,Wind Speed,Country,Datetime
30,acapulco,16.8634,-99.8901,73.76,57.0,8,4.83,MX,1648187246
83,illapel,-31.6308,-71.1653,66.83,45.0,0,1.03,CL,1648187365
252,ormara,25.2088,64.6357,86.31,34.0,0,3.04,PK,1648187990
308,iranshahr,27.2025,60.6848,87.30,11.0,0,1.16,IR,1648188077
325,faya,18.3851,42.4509,66.24,21.0,1,1.77,SA,1648188102
410,ankazoabo,-22.2833,44.5167,81.00,66.0,3,4.25,MG,1648188212
418,muzaffarabad,34.3700,73.4711,79.25,24.0,0,1.83,PK,1648188226
456,pishin,30.5803,66.9961,68.52,15.0,0,0.87,PK,1648188281
496,ontario,34.0633,-117.6509,72.54,45.0,0,3.44,US,1648188337


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#Created a hotel dataframe
#Added a column for hotels
hotel_df = ideal_df
hotel_df['Hotel Name'] = ""
hotel_df

C:\Users\NAVID\AppData\Local\Temp/ipykernel_18792/433058523.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Latitude,Longitude,Max Temp (F),Humidity,Cloud Coverage,Wind Speed,Country,Datetime,Hotel Name
30,acapulco,16.8634,-99.8901,73.76,57.0,8,4.83,MX,1648187246,
83,illapel,-31.6308,-71.1653,66.83,45.0,0,1.03,CL,1648187365,
252,ormara,25.2088,64.6357,86.31,34.0,0,3.04,PK,1648187990,
308,iranshahr,27.2025,60.6848,87.30,11.0,0,1.16,IR,1648188077,
325,faya,18.3851,42.4509,66.24,21.0,1,1.77,SA,1648188102,
410,ankazoabo,-22.2833,44.5167,81.00,66.0,3,4.25,MG,1648188212,
418,muzaffarabad,34.3700,73.4711,79.25,24.0,0,1.83,PK,1648188226,
456,pishin,30.5803,66.9961,68.52,15.0,0,0.87,PK,1648188281,
496,ontario,34.0633,-117.6509,72.54,45.0,0,3.44,US,1648188337,


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
# Creating my google map with a new layer called Marker_layer that will include the info_box content above
fig = gmaps.figure()
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, 
                                 info_box_content=hotel_info)

# Display figure
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))